In [1]:
import itertools
from pprint import pprint

In [2]:

def grid_search(grid: dict[str, list]):
    """Takes a dictionary of lists. Each list is a parameter to search over.
    The search/test space is the cartesian product of all the lists.
    """
    product = list(itertools.product(*grid.values()))
    for i, params in enumerate(product):
        data = dict(zip(grid.keys(), params))
        print(f"========== Infer by Grid Search [{i+1}/{len(product)}] ==========")
        pprint(data)
        print("================================================")



# `random_joints` Model

## TEST 1: Benchmark-Clip with IK Methods and Editable Features 

In [3]:
test_grid_1 = {
    "bvh_path": ["dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"],
    "edit_mode": ["benchmark_clip"],
    "num_samples": [1],
    "foot_ik": [True, False],
    "jacobian_ik": [True, False],
    "editable_features": ["pos_rot", "pos_rot_vel"],
}

grid_search(test_grid_1)

========== Infer by Grid Search [1/8] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot',
 'foot_ik': True,
 'jacobian_ik': True,
 'num_samples': 1}
========== Infer by Grid Search [2/8] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': True,
 'jacobian_ik': True,
 'num_samples': 1}
========== Infer by Grid Search [3/8] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot',
 'foot_ik': True,
 'jacobian_ik': False,
 'num_samples': 1}
==

### Results 

- `IK` Methoden
  - `jacobian` geht kaputt beim Umdrehen
  - `basic` Root/Legs flippen kurz bevor der Charakter bei z=0 stehen bleibt
- `editable_features`
  - `pos` geht kaputt beim Umdrehen
  - `pos_rot` 
    - innerhalb t_obs hängts hinterher
    - innerhalb t_sample stimmt die Root Position ABER Footsliding
  - `pos_rot_vel` 
    - innerhalb t_obs stimmt die Root Position
    - innerhalb t_sample stimmt hängts etwas hinterher dafür KEIN Footsliding
- `foot_ik` bringt gar nicht soo viel
  - Rutscht trotzdem, vllt lohnt es sich mal den Threshold zu erhöhen
  - Alternativ mal die Foot Contact Flags zu benutzen die das Modell vorhersagt
 
 ### Takeaways

- [ ] Basic IK Flipping fixen
  - [ ] Evtl reicht hier ein Euler Filter / Euler Unrolling
- [ ] Jacobian hat evtl ein hardcoded forward Vector?
- [ ] Methoden überlegen wie die Velocity gefaked werden kann bei einzelnen Keyframes
  - [ ] Evtl doch mal statt `stepped` auch einen `linear` Initializer probieren
- [ ] Foot IK Threshold erhöhen und auch mal die Foot Contact Flags benutzen


## TEST 2: Benchmark-Clip with Imputation

In [4]:
test_grid_2 = {
    "bvh_path": [
        "dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"
    ],
    "edit_mode": ["benchmark_clip"],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot_vel"],

    "imputate": [True],
    "stop_imputation_at": [0, 1, 10, 100],
}

grid_search(test_grid_2)

========== Infer by Grid Search [1/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'imputate': True,
 'jacobian_ik': False,
 'num_samples': 1,
 'stop_imputation_at': 0}
========== Infer by Grid Search [2/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'imputate': True,
 'jacobian_ik': False,
 'num_samples': 1,
 'stop_imputation_at': 1}
========== Infer by Grid Search [3/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip

### Results

- `stop_imputation_at=0`
  - innerhalb t_obs: perfekt
  - innerhalb t_sample: Root Sprung nach hinten, Rotationen aber okay
- `stop_imputation_at=1`
  - innerhalb t_obs: leicht hinterher
  - innerhalb t_sample: 
    - fast deckungsgleich mit `0`
    - aber halt ohne den Sprung weil t_obs schon leicht hintendran ist und sich der Offset langsam auf und abbaut
- `stop_imputation_at=10`
  - innerhalb t_obs: wie bei `1`
  - innerhalb t_sample: wie bei `1` mit etwas mehr Offset
- `stop_imputation_at=100`
  - innerhalb t_obs: wie bei `10`
  - innerhalb t_sample: sehr ähnlich aber leicht besser als `10`

### Takeaways

- Imputate mit Stop 0 ist wegen des Sprungs unbrauchbar
- Imputate mit Stop >= 1 könnte in Sonderfällen interessant sein
  - [ ] Mal bei `benchmark_sparse` testen?

## TEST 3: Benchmark-Clip with Reconstruction Guidance

In [5]:
test_grid_3 = {
    "bvh_path": [
        "dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"
    ],
    "edit_mode": ["benchmark_clip"],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot_vel"],

    "reconstruction_guidance": [True],
    "stop_recguidance_at": [0, 1, 10],
    "reconstruction_weight": [1, 5, 20],
}

grid_search(test_grid_3)

========== Infer by Grid Search [1/9] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1,
 'reconstruction_guidance': True,
 'reconstruction_weight': 1,
 'stop_recguidance_at': 0}
========== Infer by Grid Search [2/9] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1,
 'reconstruction_guidance': True,
 'reconstruction_weight': 5,
 'stop_recguidance_at': 0}
========== Infer by Grid Search [3/9] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_aroun

### Results
- `stop=0`
  - `w=1`
    - innerhalb t_obs: Nah an Original, leicht dahinter
    - innerhalb t_sample: Leichter Sprung Richtung Original
  - `w=5`
    - innerhalb t_obs: Nah an Original, leicht dahinter
    - innerhalb t_sample: Merklicher Sprung Richtung Original
  - `w=10`
    - innerhalb t_obs: Nah an Original, leicht dahinter
    - innerhalb t_sample: Overshoot und Moonwalk 
- `stop=1`
  - Kein Sprung mehr
  - Höhere `w` Werte sind näher an Original, aber beinahe unmerklich
- `stop=10`
  - Unmerklicher Unterschied zu `stop=1`


### Takeaways

- Auch hier ist `stop=0` unbrauchbar wegen des Sprungs

## TEST 4: Benchmark-Clip with Reconstruction Guidance and Imputation


In [6]:
test_grid_5 = {
    "bvh_path": [
        "dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"
    ],
    "edit_mode": ["benchmark_clip"],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot_vel"],

    "reconstruction_guidance": [True],
    "stop_recguidance_at": [1],
    "reconstruction_weight": [1, 5],

    "imputate": [True],
    "stop_imputation_at": [1],
}

grid_search(test_grid_5)

========== Infer by Grid Search [1/2] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'imputate': True,
 'jacobian_ik': False,
 'num_samples': 1,
 'reconstruction_guidance': True,
 'reconstruction_weight': 1,
 'stop_imputation_at': 1,
 'stop_recguidance_at': 1}
========== Infer by Grid Search [2/2] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'imputate': True,
 'jacobian_ik': False,
 'num_samples': 1,
 'reconstruction_guidance': True,
 'reconstruction_weight': 5,
 'stop_imputation_at': 1,
 'stop_recguidance_at': 1}


### Results

- Beides zusammen liefert keinen Mehrwert
- `w=5` ist besser als `w=1` (wie erwartet)

- Inferenz mit beidem zusammen dauert 3:30 statt 1:50 min ...


| Method     | Time |
|------------|------|
| Normal     | 1:50 |
| Imputation | 1:50 |
| Recguide   | 3:18 |
| Both       | 3:30 |

## TEST 5: benchmark_sparse & sparse keyframes via packed_motion

In [7]:
test_grid_5a = {
    "bvh_path": [
        "dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"
    ],
    "edit_mode": ["benchmark_sparse"],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot", "pos_rot_vel"],
}
test_grid_5b = {
    "packed_motion": [None],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot", "pos_rot_vel"],
}

grid_search(test_grid_5a)
grid_search(test_grid_5b)

========== Infer by Grid Search [1/2] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_sparse',
 'editable_features': 'pos_rot',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1}
========== Infer by Grid Search [2/2] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_sparse',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1}
========== Infer by Grid Search [1/2] ==========
{'editable_features': 'pos_rot',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1,
 'packed_motion': None}
========== Infer by Grid Search [2/2] ==========
{'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1,
 'packed_mot

### Results

- `benchmark_sparse`
  - Bei `pos_rot` bleibt der Character sehr weit hinterher
  - Bei `pos_rot_vel` passts ganz gut
- `packed_motion`
  - `pos_rot` besser als `pos_rot_vel`
  - `pos_rot_vel` schwimmt bzw hat Footsliding

### Takeaways

- [ ] Definitiv eine Root Trajectory als Input anbieten, ggf einfach linear interpolieren?
- [ ] Linear statt Stepped Initializer probieren
- [ ] Nachschauen, ob beim Training der Feature-Mask==0 Bereich auch auf 0 gesetzt wird oder trotzdem noch Werte enthält

# `random_frames` Model

## TEST 6: random_frames Model w/ `benchmark_clip`, `benchmark_sparse` and `packed_motion` 

In [12]:
test_grid_6 = {
    "bvh_path": [
        "dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh"
    ],
    "edit_mode": ["benchmark_clip", "benchmark_sparse"],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot", "pos_rot_vel"],
}
test_grid_5b = {
    "packed_motion": [None],
    "num_samples": [1],
    "foot_ik": [False],
    "jacobian_ik": [False],
    "editable_features": ["pos_rot", "pos_rot_vel"],
}

grid_search(test_grid_6)

========== Infer by Grid Search [1/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1}
========== Infer by Grid Search [2/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_clip',
 'editable_features': 'pos_rot_vel',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples': 1}
========== Infer by Grid Search [3/4] ==========
{'bvh_path': 'dataset/HumanML3D/bvh/test/001969_fromjoint100_a_man_walks_forward_then_turns_around_and_walks_back_before_facing_back_and_standing_still.bvh',
 'edit_mode': 'benchmark_sparse',
 'editable_features': 'pos_rot',
 'foot_ik': False,
 'jacobian_ik': False,
 'num_samples'

### Results

- `random_frames > random_joints` bei vollen Keyframes
  - Weniger Abstand zum Original, dafür Zittern bei sparse Keyframes
- `random_joints > random_frames` bei sparse Keyframes
  - Mehr Abstand zum Original, dafür kein Zittern bei sparse Keyframes

- `pos_rot` haengt immer hinterher

- `benchmark_clip`
  - `pos_rot` slided komplett durch den Maskierten Bereich
  - `random_frames` 
    - besseres Alignment mit Original
    - fällt nicht zurück
    - ruhigeres Verhalten bei Stillstand
  - `random_joints` 
    - auch gutes Alignment mit Original 
    - fällt zurück sobald ausmaskiert
    - mehr Zittern bei Stillstand

- `mybenchclip` (entspricht `benchmark_clip` aber via `packed_motion`)
  - `random_frames`
    - Erst Footsliding, dann großer Fallschritt nach vorn
  - `random_joints`
    - Ein weiterer Schritt, dann Footsliding

- `benchmark_clip` vs. `mybenchclip` (via `packed_motion`)
  - `benchmark_clip` hat definitiv einen ungewollten Einfluss $>0$ auf die maskierten Bereiche
  - `mybenchclip` hat keinen Einfluss auf die maskierten Bereiche (wie gewollt!)
  - `mybenchclip` braucht irgendeine Velocity um die Lücken zu schließen, sonst gibts Footsliding
  - 

### Takeaways

- [ ] Experimente von oben (teilweise) wiederholen mit packed_motion_clip statt benchmark_clip
  - [ ] Automatisieren: Export von 3 Varianten der packed motion (sparse_keyframes, full_keyframes, clip), dann an den InferenceWorker hängen und auf dem Cluster laufen lassen
